
`## MWE




   

# Command spliter
This utility module is used to split a series of sos command into different genes to accomodate the need of AWS.
## Input
1. sos_command_file: A text file containing variouse sos command in the correct order. The output of command generator for 1 analysis is a suitable input as long as the order of operation are correct. It is assums that all the input sos command has a parameter called --region-name and the analysis can be partitioned by this parameter.
2. region_list: A table where the last column is the name of the region to be split analysis.
4. virtual_machine_path: The path of the file and working directories in the to be established virtual machine.
5. wildcard_file (WIP, please ignore)
6. output_file: The surfix of the output script.
7. aws_command_file: A text file containing the command template for mm_jobman.sh, see https://github.com/rfeng2023/mmcloud/pull/13
8. genotype_list: list of file for the genotype file per chrom, regardless of what path you put in the genotype_chrom_list, only the actual file name will be used. The relative path are to be specified in the vertual machine path  parameter.


## Output
1. a set of all the sos run command in the sos command_file, in the forms of for each regions in the region list within each chrom. Each row is a region. Each file is a chromosome. The `--genoFile {} --` patteren within each command will be replaced with the genotype file of that specific chromosome per the instruction of the genotype file
   
    `{command 1} --region-name {gene_name} && {command 2} --region-name {gene_name}`
## MWE



In [ ]:
sos run command_spliter.ipynb split_command --sos_command_file test_sos_command --aws_command_file test_aws_command \
    --genotype_list /sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.txt \
    --virtual_machine_path "analysis/" \
    --region_list  test_command_region_list

In [ ]:
[global]
parameter: sos_command_file = path
parameter: aws_command_file = path
parameter: wildcard_file = path("")
#parameter: s3_path = str
parameter: virtual_machine_path = str
parameter: region_list = path
parameter: cwd = path(".")
parameter: output_file = path("splited_cripts.sh")
parameter: genotype_list = path
import pandas as pd
chrom = list(set(pd.read_csv(region_list,sep ="\t")["#chr"]))

[split_command_1]
input: sos_command_file, for_each = "chrom"
output: f'{cwd}/{_input:bn}.{_chrom}.sos_splited_script'

import pandas as pd
import re
region = pd.read_csv(region_list,sep = "\t")
region = region[region.iloc[:, 0] == _chrom].iloc[:,-1].tolist() ## Subset the region
genotype_list = pd.read_csv(genotype_list,sep = "\t")
per_chrom_genoFile = genotype_list [genotype_list.iloc[:, 0].astype("str").replace("chr","") == str(_chrom).replace("chr","")].iloc[0,-1]
per_chrom_genoFile = per_chrom_genoFile.split("/")[-1]

# Step 1: Read and modify the commands from a text file
commands = []
with open(_input, 'r') as file:
    command = ""
    for line in file:
        stripped_line = line.strip()
        if stripped_line.startswith('sos run'):
            if command:
                # Replace text between --genoFile and -- using regular expression
                command = re.sub(r'(--genoFile ).*?( --)', r'\1' + f'{virtual_machine_path}/{per_chrom_genoFile}' + r'\2', command)
                commands.append(command)
            command = stripped_line
        elif command.endswith('\\'):
            command = command[:-1].strip() + ' ' + stripped_line
    if command:
        # Replace text in the last command
        command = re.sub(r'(--genoFile ).*?( --)', r'\1'  + f'{virtual_machine_path}/{per_chrom_genoFile}'  + r'\2', command)
        commands.append(command)

# Step 2: Create a DataFrame with region names appended to each command
data = {command: [f"{command} --region-name {r}" for r in region] for command in commands}
df_commands = pd.DataFrame(data, index=region)

# Step 3: Creating the concatenated list
concatenated_commands = [" && ".join(row) for _, row in df_commands.iterrows()]

# Define your paths for the copy command
path2 = virtual_machine_path

# Step 4: Modifying each command
modified_commands = [f" {cmd} " for cmd in concatenated_commands]

output_file = f'{modified_commands}.sh'
with open(_output, 'w') as f:
    for cmd in modified_commands:
        f.write(cmd + '\n')

[split_command_2]
input: group_with = "chrom"
output: f'{cwd}/{_input:bn}.jobman.sh'
genotype_list = pd.read_csv(genotype_list,sep = "\t")
per_chrom_genoFile = genotype_list[genotype_list.iloc[:, 0].astype("str").replace("chr","") == str(_chrom).replace("chr","")].iloc[0,-1]
per_chrom_genoFile = per_chrom_genoFile.split("/")[-1].rsplit('.', 1)[0]

with open(aws_command_file, 'r') as file:
    lines = file.readlines()
    
    # Identify the index of the second non-empty line
second_non_empty_line_index = [i for i, line in enumerate(lines) if line.strip()][2]


# Modify the second non-empty line
lines[second_non_empty_line_index] = f"./{_input} \\\n"

# Append to the last line
lines[-1] = lines[-1].strip() + f'  --download-include {per_chrom_genoFile}.bed {per_chrom_genoFile}.bim {per_chrom_genoFile}.fam \n'

# Write the modified content to a new file
with open(_output, 'w') as file:
    file.writelines(lines)

[split_command_3]
input: group_by = "all"
output: f'{cwd}/{sos_command_file:bn}.{aws_command_file:bn}.all_chrom.jobman.sh'
# List of file paths to concatenate
# Open the new file in write mode
with open(_output, 'w') as new_file:
    # Iterate through each file in the list
    for file_path in _input:
        # Open and read each file
        with open(file_path, 'r') as file:
            # Write the content to the new file
            new_file.write(file.read())
            # Optionally add a newline between files
            new_file.write('\n')
